<H3>Este programa preenche valores de preços ausentes em uma planilha Excel usando interpolação linear.</H3>

In [2]:
import pandas as pd 
import numpy as np
from datetime import datetime
import locale

<h4>O programa já pede o caminho da planilha ao ser executado, o trecho abaixo é apenas para mostrar os valores faltantes da tabela</h4>

In [ ]:
df = pd.read_excel("dados_ficticios.xlsx")
df

,data do preco,preco produto
0,2022-01-01,125.40
1,2022-02-01,128.65
2,2022-03-01,132.20
3,2022-04-01,142.75
4,2022-05-01,148.30
5,2022-06-01,155.80
6,2022-07-01,162.45
7,2022-08-01,NaN
8,2022-09-01,155.35
9,2022-10-01,145.20


In [25]:
try:
    locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')
except:
    try:
        locale.setlocale(locale.LC_ALL, 'Portuguese_Brazil.1252')
    except:
        pass  

def converter_para_numero(valor_str):
    """Converte string de valor no formato brasileiro para float"""
    if pd.isna(valor_str) or valor_str == '':
        return np.nan
    
    valor_limpo = str(valor_str).replace('R$', '').strip()
    
    if ',' in valor_limpo:
        valor_limpo = valor_limpo.replace('.', '').replace(',', '.')
    
    try:
        return float(valor_limpo)
    except ValueError:
        return np.nan

def formatar_valor_brasileiro(valor):
    """Formata um número para o padrão brasileiro de moeda"""
    if pd.isna(valor):
        return ""
    
    return f"R$ {valor:.2f}".replace('.', ',')

def preencher_precos_faltantes(df):
    """
    Preenche valores faltantes de preço e padroniza todos os valores para o formato
    brasileiro de moeda (R$ com vírgula como separador decimal)
    """
    df_resultado = df.copy()
    
    df_resultado['DATA'] = pd.to_datetime(df_resultado['data do preco'], errors='coerce')
    df_resultado['PRECO_NUMERICO'] = df_resultado['preco produto'].apply(converter_para_numero)
    df_resultado['preco produto'] = df_resultado['preco produto'].astype('object')
    
    df_resultado = df_resultado.sort_values(by=['DATA'])
    df_resultado['PREENCHIDO'] = False
    
    valores_faltantes_antes = df_resultado['PRECO_NUMERICO'].isna().sum()
    print(f"Valores faltantes antes da interpolação: {valores_faltantes_antes}")
    
    indices_faltantes = df_resultado[df_resultado['PRECO_NUMERICO'].isna()].index.tolist()
    
    for idx in indices_faltantes:
        data_faltante = df_resultado.loc[idx, 'DATA']
        df_validos = df_resultado[~df_resultado['PRECO_NUMERICO'].isna()].copy()
        
        if len(df_validos) < 2:
            print("Não há dados suficientes para interpolação.")
            continue
        
        registros_anteriores = df_validos[df_validos['DATA'] < data_faltante]
        if len(registros_anteriores) == 0:
            print(f"Aviso: Extrapolando valor para a data {data_faltante}")
            continue
            
        registro_anterior = registros_anteriores.iloc[-1]
        data_anterior = registro_anterior['DATA']
        preco_anterior = registro_anterior['PRECO_NUMERICO']
        
        registros_posteriores = df_validos[df_validos['DATA'] > data_faltante]
        if len(registros_posteriores) == 0:
            print(f"Aviso: Extrapolando valor para a data {data_faltante}")
            continue
            
        registro_posterior = registros_posteriores.iloc[0]
        data_posterior = registro_posterior['DATA']
        preco_posterior = registro_posterior['PRECO_NUMERICO']
        
        dias_total = (data_posterior - data_anterior).days
        dias_proporcao = (data_faltante - data_anterior).days
        
        if dias_total == 0:
            preco_interpolado = (preco_anterior + preco_posterior) / 2
        else:
            preco_interpolado = preco_anterior + (preco_posterior - preco_anterior) * (dias_proporcao / dias_total)
        
        df_resultado.loc[idx, 'PRECO_NUMERICO'] = preco_interpolado
        df_resultado.loc[idx, 'preco produto'] = formatar_valor_brasileiro(preco_interpolado)
        df_resultado.loc[idx, 'PREENCHIDO'] = True
    
    valores_preenchidos = df_resultado['PREENCHIDO'].sum()
    print(f"Valores preenchidos pela interpolação: {valores_preenchidos}")
    
    for idx in df_resultado.index:
        if not pd.isna(df_resultado.loc[idx, 'PRECO_NUMERICO']):
            df_resultado.loc[idx, 'preco produto'] = formatar_valor_brasileiro(df_resultado.loc[idx, 'PRECO_NUMERICO'])
    
    return df_resultado

def main():
    try:
        print("Carregando arquivo Excel...")
        arquivo_excel = input("Digite o caminho do arquivo Excel: ")
        df = pd.read_excel(arquivo_excel)
        
        print(f"Arquivo carregado com sucesso. {len(df)} registros encontrados.")
        print(f"Colunas encontradas: {', '.join(df.columns)}")
        
        colunas_necessarias = ['data do preco', 'preco produto']
        for coluna in colunas_necessarias:
            if coluna not in df.columns:
                print(f"Erro: Coluna '{coluna}' não encontrada na planilha.")
                return
        
        print("Preenchendo valores faltantes...")
        df_preenchido = preencher_precos_faltantes(df)
        
        colunas_para_remover = ['PRECO_NUMERICO', 'DATA']
        df_final = df_preenchido.drop(columns=colunas_para_remover)
        
        arquivo_saida = arquivo_excel.replace('.xlsx', '_preenchido.xlsx')
        if arquivo_saida == arquivo_excel:
            arquivo_saida = arquivo_excel.replace('.xls', '_preenchido.xlsx')
        if arquivo_saida == arquivo_excel:
            arquivo_saida = "resultado_preenchido.xlsx"
            
        print(f"Salvando resultado em '{arquivo_saida}'...")
        
        with pd.ExcelWriter(arquivo_saida, engine='openpyxl') as writer:
            df_final.to_excel(writer, index=False, sheet_name='Dados Preenchidos')
            
            for column in df_final:
                column_width = max(df_final[column].astype(str).map(len).max(), len(column))
                col_idx = df_final.columns.get_loc(column)
                writer.sheets['Dados Preenchidos'].column_dimensions[chr(65 + col_idx)].width = column_width + 2
        
        print("Processo concluído com sucesso!")
        print(f"Total de registros originais: {len(df)}")
        print(f"Total de registros no resultado: {len(df_final)}")
        
    except Exception as e:
        import traceback
        print(f"Erro: {str(e)}")
        print(traceback.format_exc())

if __name__ == "__main__":
    main()

Carregando arquivo Excel...
Arquivo carregado com sucesso. 30 registros encontrados.
Colunas encontradas: data do preco, preco produto
Preenchendo valores faltantes...
Valores faltantes antes da interpolação: 3
Valores preenchidos pela interpolação: 3
Salvando resultado em 'C:\Users\jonat\.vscode\python-ws\projecoes\dados_ficticios_preenchido.xlsx'...
Processo concluído com sucesso!
Total de registros originais: 30
Total de registros no resultado: 30


In [26]:
df_interpolado = pd.read_excel("dados_ficticios_preenchido.xlsx")
df_interpolado

,data do preco,preco produto,PREENCHIDO
0,2022-01-01,"R$ 125,40",False
1,2022-02-01,"R$ 128,65",False
2,2022-03-01,"R$ 132,20",False
3,2022-04-01,"R$ 142,75",False
4,2022-05-01,"R$ 148,30",False
5,2022-06-01,"R$ 155,80",False
6,2022-07-01,"R$ 162,45",False
7,2022-08-01,"R$ 158,90",True
8,2022-09-01,"R$ 155,35",False
9,2022-10-01,"R$ 145,20",False
